In [1]:
## Author: Mirjam Nanko
## Date Created: 2022-02-22
## Email: m.nanko@exeter.ac.uk

# socialCARDS classifier training and evaluation<br>
#### This script loads the **training, validation and testing** data sourced from various **facebook and twitter accounts** and trains a classifier to classify these into **convinced** (0) and **contrarian** (1).

# Packages

In [2]:
from preprocess import denoise_text
from logistic import fit_logistic_classifier
import pandas as pd
from sklearn import metrics
import pickle
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='bs4')

# Load and preprocess the training, validation and testing data

In [3]:
%%time
# Load the data
train = pd.read_csv('../data/socialCARDStrain.csv', lineterminator='\n')
valid = pd.read_csv('../data/socialCARDSvalid.csv', lineterminator='\n')
test = pd.read_csv('../data/socialCARDStest.csv', lineterminator='\n')
# Preprocess the text
train['text'] = train['text'].astype(str).apply(denoise_text)
valid['text'] = valid['text'].astype(str).apply(denoise_text)
test['text'] = test['text'].astype(str).apply(denoise_text)

CPU times: user 4min 22s, sys: 7.9 s, total: 4min 30s
Wall time: 4min 28s


# Train the logistic classifier

In [4]:
%%time
# Fit the classifier
data = train[['text','label']].values.tolist()
model = fit_logistic_classifier(data)

CPU times: user 45min 3s, sys: 1min 18s, total: 46min 21s
Wall time: 6min 40s


# Encode and vectorize the validation and testing data

In [5]:
%%time
# Encode the labels
y_valid = model['label_encoder'].transform(valid.label)
y_test = model['label_encoder'].transform(test.label)
# Vectorize the text
X_valid = model['vectorizer'].transform(valid.text)
X_test = model['vectorizer'].transform(test.text)

CPU times: user 45 s, sys: 872 ms, total: 45.8 s
Wall time: 45.8 s


# Inspect the classifier performance

In [6]:
print('Validation data:\n')
print(metrics.classification_report(y_valid,model['clf'].predict(X_valid)))
print('\n\nTesting data (unseen twitter and facebook accounts):\n')
print(metrics.classification_report(y_test,model['clf'].predict(X_test)))

Validation data:

              precision    recall  f1-score   support

           0       0.88      0.90      0.89    484130
           1       0.91      0.89      0.90    547607

    accuracy                           0.90   1031737
   macro avg       0.90      0.90      0.90   1031737
weighted avg       0.90      0.90      0.90   1031737



Testing data (unseen twitter and facebook accounts):

              precision    recall  f1-score   support

           0       0.70      0.90      0.79    166797
           1       0.89      0.67      0.77    200079

    accuracy                           0.78    366876
   macro avg       0.79      0.79      0.78    366876
weighted avg       0.80      0.78      0.78    366876



# Save the classifier

In [7]:
# Save the classifier
pkl_filename = "../classifiers/socialCARDS.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(model, file)
# Load classifier from file
# with open(pkl_filename, 'rb') as file:
#     model = pickle.load(file)